In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Abc123"
host = "nv-desktop-services.apporto.com"
port = 31630  
db_name = "AAC"  
collection = "animals"  
shelter = AnimalShelter(username, password, host, port, db_name, collection)

# Connect to database via CRUD Module
# db = AnimalShelter(username, password)
# Function to fetch data by rescue type for dynamic filtering
def get_dogs_by_rescue_type(rescue_type):
    queries = {
        "Water": {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
        "Mountain or Wilderness": {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
        "Disaster": {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    }
    query = queries.get(rescue_type, {})
    results = shelter.read(query)
    return pd.DataFrame.from_records(results).drop(columns=['_id'], errors='ignore')


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Logo-Copy1.png' #saved in dessktop as Logo-Copy1.png
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div([
        html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'50px', 'width':'auto'})], href='http://www.snhu.edu', target='_blank'),
        html.Div('Dashboard created by Jacob Batrano', style={'float': 'right', 'color': '#007BFF'})
    ]),
    html.Center(html.B(html.H1('CS-340 Dashboard for Grazioso Salvare'))),
    html.Hr(),
    html.Div(
        
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='All',
            clearable=False
        ),
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=20,  # Show 20 rows per page
                         style_table={'height': '300px', 'overflowY': 'auto', 'overflowX': 'auto'},
                         style_cell={'minWidth': '100px', 'width': '150px', 'maxWidth': '200px', 'whiteSpace': 'normal'},
                         sort_action="native",  # Enables sorting on all columns
                         filter_action="native",  # Enables filtering on all columns
                         editable=True,  # Allow editing of data inside the table
                         row_selectable='single',  # Enable single row selection
                         selected_rows=[0]
                         ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type in ['All', 'Reset']:
        df_filtered = pd.DataFrame.from_records(shelter.read({})).drop(columns=['_id'], errors='ignore')
    else:
        df_filtered = get_dogs_by_rescue_type(filter_type)
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame(viewData)
    if 'breed' in dff.columns:
        fig = px.pie(dff, names='breed', title='Distribution of Breeds')
        pie_chart = dcc.Graph(figure=fig)
    else:
        pie_chart = html.Div("No data available for chart.")
    return [pie_chart]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)



Dash app running on http://127.0.0.1:12827/
